In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import os

In [3]:
%mkdir ./horse
%mkdir ./zebra

In [4]:
%cp ../input/horse2zebra/horse2zebra/trainA/* ./horse 
%cp ../input/horse2zebra/horse2zebra/trainB/* ./zebra

In [5]:
%mkdir ./dataset

In [6]:
%mv ./horse ./dataset
%mv ./zebra ./dataset

In [7]:
dataset_path = os.listdir('dataset')
print(dataset_path)

print("Types of classes labels found: ", len(dataset_path))

In [8]:
class_labels = []

for item in dataset_path:
  all_classes = os.listdir('dataset' + '/' + item)

  # Add into the list
  for room in all_classes:
    class_labels.append((item, str('dataset_path' + '/' + item) + '/' + room))

In [9]:
print(class_labels[:5])

In [10]:
# Build dataframe
df = pd.DataFrame(data = class_labels, columns = ['labels', 'image'])
print(df.head())
print(df.tail())

In [11]:
df.info()

In [12]:
# Check how many samples for each category present
print("Total number of images: ", len(df))

label_count = df['labels'].value_counts()
print(label_count)

In [13]:
import cv2
path = 'dataset/'
dataset_path = os.listdir('dataset')

im_size = 240

images = []
labels = []

for i in dataset_path:
  data_path = path + str(i)
  filenames = [i for i in os.listdir(data_path)]

  for f in filenames:
    img = cv2.imread(data_path + '/' + f)
    img = cv2.resize(img, (im_size, im_size))
    images.append(img)
    labels.append(i)

In [14]:
# Model take input (240,240,3) and the input with range [0, 255]

images = np.array(images)
images = images.astype('float32') / 255.0
images.shape

In [15]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
y = df['labels'].values
print(set(y))

y_labelencoder = LabelEncoder()
y = y_labelencoder.fit_transform(y)
print(set(y))

In [16]:
y = y.reshape(-1,1)

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) # Into array() -> .toarray()
print(Y[:5])
print(Y[35:])

In [17]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images, Y = shuffle(images, Y, random_state=21)
train_X, test_X, train_y, test_y = train_test_split(images, Y, test_size=0.1, random_state=21)

# Inspect the shape of the data
print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

## EfficientNetB1

In [18]:
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras import layers

NUM_CLASSES = 2
IMG_SIZE = 240
size = (IMG_SIZE, IMG_SIZE)

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Using model without transfer learning
outputs = EfficientNetB1(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)

In [25]:
del(model)

In [26]:
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [27]:
hist = model.fit(train_X, train_y, epochs=10, verbose=2)

In [28]:
def plot_hist(hist):
  plt.plot(hist.history['accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel("epoch")
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()

plot_hist(hist)

In [29]:
preds = model.evaluate(test_X, test_y)
print('loss: ' + str(preds[0]))
print("Test Accuracy: " + str(preds[1]))

In [30]:
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from matplotlib.pyplot import imread, imshow

In [33]:
img_path = "../input/testing-data/horse.jpg"

img = cv2.imread(img_path)
img = cv2.resize(img, (240, 240))

x = np.expand_dims(img, axis=0)
x = preprocess_input(x)

print("Image shape: ", x.shape)

my_img = imread(img_path)
imshow(my_img)

In [34]:
preds = model.predict(x)
preds.round(3)

In [35]:
img_path = "../input/testing-data/horse2.jpg"

img = cv2.imread(img_path)
img = cv2.resize(img, (240, 240))

x = np.expand_dims(img, axis=0)
x = preprocess_input(x)

print("Image shape: ", x.shape)

my_img = imread(img_path)
imshow(my_img)

In [36]:
preds = model.predict(x)
preds.round(3)

In [37]:
img_path = "../input/testing-data/zebra.jpg"

img = cv2.imread(img_path)
img = cv2.resize(img, (240, 240))

x = np.expand_dims(img, axis=0)
x = preprocess_input(x)

print("Image shape: ", x.shape)

my_img = imread(img_path)
imshow(my_img)

In [38]:
preds = model.predict(x)
preds.round(3)